In [1]:
from datasets import load_dataset, Dataset
import torch

/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ds = load_dataset("phiyodr/coco2017")

In [4]:
# ds.save_to_disk("../coco2017_dataset")

In [3]:
from datasets import load_from_disk

!pwd

ds = load_from_disk("../../../datasets/coco2017_dataset")
ds = ds['validation']
ds

/home/elena/emcomm/emcomm_captions/notebooks/draft_and_situational


Dataset({
    features: ['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'image_id', 'ids', 'captions'],
    num_rows: 5000
})

In [4]:
ds = ds.select_columns(['coco_url', 'captions', 'image_id'])
ds

Dataset({
    features: ['coco_url', 'captions', 'image_id'],
    num_rows: 5000
})

In [7]:
# def load_image(example):
#     from PIL import Image
#     import requests
#     from io import BytesIO

#     response = requests.get(example['coco_url'])
#     example['image'] = Image.open(BytesIO(response.content))
#     return example


# ds = ds.map(load_image, num_proc = 64)
# ds

In [8]:
# def load_images(batch):
#     import requests
#     from PIL import Image
#     from io import BytesIO

#     images = []
#     for url in batch["coco_url"]:
#         r = requests.get(url)
#         images.append(Image.open(BytesIO(r.content)))

#     batch["image"] = images
#     return batch

# ds = ds.with_transform(load_images)
# ds

In [5]:
print(torch.cuda.device_count())
print([torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])

2
['NVIDIA TITAN RTX', 'NVIDIA TITAN RTX']


In [6]:
from torchvision import models, transforms

# Load ResNet model and set to eval mode
resnet = models.resnet152(pretrained=True)
resnet.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet.to(device)
# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# def image_to_features(example):
#     # Convert PIL image to tensor and preprocess
#     image = example["image"]
#     input_tensor = preprocess(image).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
#     resnet.to("cuda" if torch.cuda.is_available() else "cpu")
#     with torch.no_grad():
#         features = resnet(input_tensor)
#     return {"image_tensor": features.cpu().squeeze().numpy()}


def images_to_features(batch):
    """
    batch: dictionary with key 'image' containing a list of PIL images
    Returns: dictionary with key 'features' containing a list of numpy arrays
    """
    inputs = torch.stack([preprocess(img) for img in batch["image"]]).to(device)
    resnet.to(device)
    with torch.no_grad():
        outputs = resnet(inputs)
    # Convert to CPU numpy arrays
    batch["features"] = [feat.cpu().numpy() for feat in outputs]
    return batch


/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
def load_image_and_features(example):
    from PIL import Image
    import requests
    from io import BytesIO

    response = requests.get(example['coco_url'])
    image = Image.open(BytesIO(response.content))
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    resnet.to(device)
    with torch.no_grad():
        features = resnet(input_tensor)
    return {"image_features": features}

In [9]:
ds_small = ds.select(range(1000))

In [14]:
new_ds = ds_small.map(
    load_image_and_features,
    # batched=True,
    # batch_size=16,  # adjust based on your GPU memory
    remove_columns=[col for col in ds.column_names if col not in ['image_features', "captions", "coco_url", "image_id"]],
    # num_proc=1
)
new_ds

Map: 100%|██████████| 1000/1000 [11:52<00:00,  1.40 examples/s]


Dataset({
    features: ['coco_url', 'captions', 'image_id', 'image_features'],
    num_rows: 1000
})

In [15]:
new_ds['image_features'][0]

[[-1.5541355609893799,
  -1.5947059392929077,
  -1.1687434911727905,
  -1.8138822317123413,
  -3.180978775024414,
  -4.795310974121094,
  -2.165529727935791,
  1.702135682106018,
  -1.6198885440826416,
  0.9179667830467224,
  -1.9585742950439453,
  -2.19325590133667,
  -1.642240047454834,
  -0.9938718676567078,
  -1.4040323495864868,
  -0.6884080171585083,
  -0.7003340125083923,
  0.04388124868273735,
  0.041800014674663544,
  -0.288299560546875,
  -2.0193350315093994,
  -1.578482985496521,
  -1.9804953336715698,
  -1.3179528713226318,
  -0.16999617218971252,
  -2.538370132446289,
  -1.4389619827270508,
  -1.8231326341629028,
  -2.4049181938171387,
  -2.9735708236694336,
  -3.280374765396118,
  -1.8101320266723633,
  -3.711230754852295,
  -1.2693464756011963,
  0.08133243769407272,
  0.8075945973396301,
  1.633764624595642,
  0.08163582533597946,
  -4.505173206329346,
  0.05726019665598869,
  -2.9467480182647705,
  -0.9676252007484436,
  2.1032874584198,
  0.930618166923523,
  -0.25656

In [14]:
def fetch_image(url: str):
    from PIL import Image
    import requests
    from io import BytesIO
    r = requests.get(url, timeout=10)
    return Image.open(BytesIO(r.content)).convert("RGB")


def load_and_extract(batch):
    from concurrent.futures import ThreadPoolExecutor
    # 1. Parallel download (I/O bound → threads are perfect)
    with ThreadPoolExecutor(max_workers=24) as pool:
        images = list(pool.map(fetch_image, batch["coco_url"]))

    # 2. Batch preprocessing
    inputs = torch.stack([preprocess(img) for img in images]).to(device)

    # 3. Single-process GPU inference
    with torch.no_grad():
        outputs = resnet(inputs)

    batch["features"] = [f.cpu().numpy() for f in outputs]
    return batch

In [11]:
full_feature_ds = ds.map(
    load_and_extract,
    batched=True,
    batch_size=256,   # controls GPU batch size
    # num_proc=1       # IMPORTANT: keep this 1 (GPU safe)
)

Map: 100%|██████████| 5000/5000 [02:57<00:00, 28.18 examples/s]


In [15]:
full_feature_ds = full_feature_ds.rename_column("image_features", "features")
full_feature_ds['features'][0]

[-2.4849164485931396,
 -1.8609575033187866,
 -0.8572610020637512,
 -3.2787601947784424,
 -0.46256136894226074,
 -0.9316834211349487,
 -0.6684622764587402,
 -0.11642013490200043,
 -1.2542853355407715,
 -2.591439723968506,
 -1.4111003875732422,
 -3.214085340499878,
 -1.6102299690246582,
 -1.059586524963379,
 -4.116660118103027,
 -1.669549584388733,
 -1.8512635231018066,
 -2.442902088165283,
 -2.0985121726989746,
 -0.5741544961929321,
 -1.200879454612732,
 -3.3121471405029297,
 -3.8707706928253174,
 -1.775404691696167,
 -2.9672250747680664,
 -2.295726776123047,
 -4.336905479431152,
 -3.39382004737854,
 -1.355884075164795,
 -2.8329572677612305,
 -1.7735704183578491,
 -3.080526113510132,
 -4.84385871887207,
 -0.9667870998382568,
 -0.013195216655731201,
 -0.342542439699173,
 1.1143138408660889,
 0.16926643252372742,
 -3.2690470218658447,
 0.30506038665771484,
 -3.2891454696655273,
 -2.7209346294403076,
 -2.017390489578247,
 -0.8689264059066772,
 -3.476243019104004,
 -0.9628586173057556,
 -3.

In [16]:
full_feature_ds.save_to_disk("../../../datasets/coco_val_features_resnet_152")

Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:00<00:00, 13137.19 examples/s]


In [ ]:
# new_ds = ds.map(
#     lambda example: {
#         "captions": example["captions"],
#         "features": torch.tensor(image_to_features(example)["image_tensor"])
#     },
#     remove_columns=[col for col in ds.column_names if col not in ["captions", 'coco_url', 'image_id']],
#     # num_proc=24
# )

In [17]:
# ds[45]['image']

In [ ]:

ds.save_to_disk("./coco_train_images")

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:02<00:00, 2163.58 examples/s] 


In [54]:
ds_train = load_from_disk("../../../datasets/coco_train_features_resnet_152")
ds_train

Dataset({
    features: ['coco_url', 'captions', 'image_id', 'features'],
    num_rows: 118287
})

In [55]:
ds_train['features'][0]

[-1.5541307926177979,
 -1.5947024822235107,
 -1.1687493324279785,
 -1.8138844966888428,
 -3.180985927581787,
 -4.795315742492676,
 -2.1655349731445312,
 1.7021362781524658,
 -1.6198887825012207,
 0.9179670810699463,
 -1.9585778713226318,
 -2.193258762359619,
 -1.6422481536865234,
 -0.9938796162605286,
 -1.4040379524230957,
 -0.688406229019165,
 -0.700332760810852,
 0.04387463629245758,
 0.041804224252700806,
 -0.28830820322036743,
 -2.019338607788086,
 -1.5784856081008911,
 -1.9804952144622803,
 -1.3179523944854736,
 -0.1699957549571991,
 -2.5383670330047607,
 -1.4389599561691284,
 -1.8231319189071655,
 -2.4049201011657715,
 -2.973569869995117,
 -3.2803781032562256,
 -1.8101356029510498,
 -3.7112343311309814,
 -1.2693501710891724,
 0.08133000135421753,
 0.8075929880142212,
 1.6337623596191406,
 0.08163295686244965,
 -4.5051751136779785,
 0.05725296586751938,
 -2.9467544555664062,
 -0.9676306247711182,
 2.103285312652588,
 0.9306120872497559,
 -0.25657257437705994,
 -1.8493010997772217,

In [ ]:
from matplotlib import pyplot as plt
# plt.imshow(ds_val['image'][0])